Load Data

In [14]:
!pip install cybotrade_datasource
!pip install python-dotenv

import pandas as pd
import asyncio
import cybotrade_datasource
from datetime import datetime, timezone
from dotenv import load_dotenv
import os

import warnings as wr
wr.filterwarnings('ignore')

print(dir(cybotrade_datasource))

load_dotenv()

API_KEY = os.getenv('CYBOTRADE_API_KEY')

'''
async def main():
    data = await cybotrade_datasource.query_paginated(
        api_key=API_KEY, 
        topic='cryptoquant|btc/inter-entity-flows/miner-to-miner?from_miner=f2pool&to_miner=all_miner&window=hour', 
        start_time=datetime(year=2024, month=1, day=1, tzinfo=timezone.utc),
        end_time=datetime(year=2025, month=1, day=1, tzinfo=timezone.utc)
    )
    df = pd.DataFrame(data)
    print(df)
    

await main()
'''

'''
async def main():
    stream = await cybotrade_datasource.stream(
        api_key=API_KEY,
        topics=[
            'cryptoquant|btc/inter-entity-flows/miner-to-miner?from_miner=f2pool&to_miner=all_miner&window=hour',
            'cryptoquant|btc/market-data/liquidations?exchange=deribit&window=min',
        ],
    )
    async for msg in stream:
        print(msg)
    

await main()
'''

global_df = None 

async def get_data(topic):
    data = await cybotrade_datasource.query_paginated(
        api_key=API_KEY, 
        topic=topic, 
        limit=10000
    )
    return pd.DataFrame(data)


async def main():
    global global_df

    topics = [
        "bybit-linear|candle?symbol=BTCUSDT&interval=1m",
    ]

#topic
#"cryptoquant|btc/market-data/open-interest?exchange=bybit&window=min",
#bybit-linear|candle?interval=1m&symbol=BTCUSDT
#"cryptoquant|btc/market-data/coinbase-premium-index?window=hour",
#"cryptoquant|btc/exchange-flows/outflow?exchange=coinbase_advanced&window=hour",
#"glassnode|market/deltacap_usd?a=BTC&i=10m",

    tasks = [get_data(topic) for topic in topics]
    dataframes = await asyncio.gather(*tasks)

    for i, df in enumerate(dataframes):
        print(f"DataFrame {i+1} ({topics[i]})")
        print(df.head)
        print(df.isnull().sum())
        global_df = df

await main()


['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'cybotrade_datasource', 'query', 'query_paginated', 'stream']
DataFrame 1 (bybit-linear|candle?symbol=BTCUSDT&interval=1m)
<bound method NDFrame.head of          start_time    close     high      low     open  volume
0     1743659280000  83364.8  83405.7  83360.0  83405.7  30.697
1     1743659340000  83393.8  83393.8  83360.2  83364.8  16.294
2     1743659400000  83405.4  83432.9  83393.7  83393.8  16.983
3     1743659460000  83391.3  83405.4  83380.0  83405.4   8.574
4     1743659520000  83384.5  83406.4  83376.3  83391.3  19.058
...             ...      ...      ...      ...      ...     ...
9995  1744258980000  81782.9  81782.9  81724.0  81749.3  18.160
9996  1744259040000  81781.7  81822.9  81771.1  81782.9  25.325
9997  1744259100000  81794.0  81794.0  81766.1  81781.7  13.042
9998  1744259160000  81770.3  81794.0  81756.4  81794.0  10.477
9999  1744259220000  817

Check Data whether got null or not

In [15]:
print(global_df)

         start_time    close     high      low     open  volume
0     1743659280000  83364.8  83405.7  83360.0  83405.7  30.697
1     1743659340000  83393.8  83393.8  83360.2  83364.8  16.294
2     1743659400000  83405.4  83432.9  83393.7  83393.8  16.983
3     1743659460000  83391.3  83405.4  83380.0  83405.4   8.574
4     1743659520000  83384.5  83406.4  83376.3  83391.3  19.058
...             ...      ...      ...      ...      ...     ...
9995  1744258980000  81782.9  81782.9  81724.0  81749.3  18.160
9996  1744259040000  81781.7  81822.9  81771.1  81782.9  25.325
9997  1744259100000  81794.0  81794.0  81766.1  81781.7  13.042
9998  1744259160000  81770.3  81794.0  81756.4  81794.0  10.477
9999  1744259220000  81706.9  81775.8  81700.0  81770.3  25.565

[10000 rows x 6 columns]
